In [ ]:
# Example: Quadratic-band wall penalty with visualization

import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# Geometry: distance to segment
# -------------------------------
def point_to_segment_distance(x, a, b):
    """
    Distance from point x to line segment [a, b]
    """
    v = b - a
    w = x - a
    vv = np.dot(v, v)

    if vv == 0.0:
        return np.linalg.norm(x - a)

    t = np.dot(w, v) / vv
    t = np.clip(t, 0.0, 1.0)
    q = a + t * v
    return np.linalg.norm(x - q)

# -------------------------------
# Wall penalty (quadratic band)
# -------------------------------
def wall_penalty(x, a, b, R):
    d = point_to_segment_distance(x, a, b)
    if d <= R:
        return 0.5 * (R - d)**2
    else:
        return 0.0

# -------------------------------
# Grid setup
# -------------------------------
xmin, xmax = -5, 5
ymin, ymax = -5, 5
N = 100

xs = np.linspace(xmin, xmax, N)
ys = np.linspace(ymin, ymax, N)
X, Y = np.meshgrid(xs, ys)

# Wall definition (horizontal wall)
a = np.array([-3.0, 0.0])
b = np.array([ 3.0, 0.0])
R = 1.0  # influence radius

# -------------------------------
# Evaluate cost field
# -------------------------------
C = np.zeros_like(X)

for i in range(N):
    for j in range(N):
        x = np.array([X[i, j], Y[i, j]])
        C[i, j] = wall_penalty(x, a, b, R)

# -------------------------------
# Numerical gradient
# -------------------------------
dCy, dCx = np.gradient(C, ys, xs)

# -------------------------------
# Plot
# -------------------------------
plt.figure(figsize=(8, 6))

# Cost contours
plt.contourf(X, Y, C, levels=30, cmap="viridis")
plt.colorbar(label="Wall penalty")

# Negative gradient (descent direction)
plt.quiver(
    X[::5, ::5], Y[::5, ::5],
    -dCx[::5, ::5], -dCy[::5, ::5],
    color="white", alpha=0.7
)

# Wall segment
plt.plot([a[0], b[0]], [a[1], b[1]], "r-", linewidth=3, label="Wall")

plt.xlabel("x")
plt.ylabel("y")
plt.title("Quadratic Band Wall Penalty and Gradient Field")
plt.axis("equal")
plt.legend()
plt.show()


## Floor plans to use

Plot floor-plan

In [2]:
def plot_floor_plan(walls, start=None, goal=None):
    plt.figure(figsize=(6, 6))
    for a, b in walls:
        a = np.array(a)
        b = np.array(b)
        plt.plot([a[0], b[0]], [a[1], b[1]], "k-", linewidth=2)

    if start is not None:
        plt.plot(start[0], start[1], "go", markersize=8, label="Start")

    if goal is not None:
        plt.plot(goal[0], goal[1], "ro", markersize=8, label="Goal")

    plt.axis("equal")
    plt.grid(True)
    plt.legend()
    plt.title("Assignment Floor Plan")
    plt.show()


### Selectable Floor Plan Layouts

In [3]:
# -----------------------------------------------------------------------------
# Floor plan options for the assignment
# -----------------------------------------------------------------------------

import numpy as np

def load_floor_plan(name="baseline"):
    """
    Load a predefined floor plan.

    Available options:
      - "baseline"  : corridor + corner (recommended default)
      - "zigzag"    : narrow zig-zag corridor (oscillation test)
      - "deadend"   : dead-end with escape (local minimum trap)
      - "symmetric" : symmetric obstacle trap (pathological case)

    Returns:
      walls : list of (a, b) line segments
      start : np.array shape (2,)
      goal  : np.array shape (2,)
    """

    if name == "baseline":
        walls = [
            # Outer boundary
            ([-5, -5], [ 5, -5]),
            ([ 5, -5], [ 5,  5]),
            ([ 5,  5], [-5,  5]),
            ([-5,  5], [-5, -5]),

            # Interior corridor
            ([-1, -5], [-1,  2]),
            ([-1,  2], [ 3,  2]),
            ([ 3,  2], [ 3, -2]),
        ]
        start = [-4.0, -4.0]
        goal  = [ 4.0,  4.0]

    elif name == "zigzag":
        walls = [
            # Outer boundary
            ([-6, -4], [ 6, -4]),
            ([ 6, -4], [ 6,  4]),
            ([ 6,  4], [-6,  4]),
            ([-6,  4], [-6, -4]),

            # Zig-zag corridor
            ([-4, -3], [-1, -3]),
            ([-1, -3], [-1,  0]),
            ([-1,  0], [ 2,  0]),
            ([ 2,  0], [ 2,  3]),
            ([ 2,  3], [ 5,  3]),
        ]
        start = [-5.0, -3.5]
        goal  = [ 5.0,  3.5]

    elif name == "deadend":
        walls = [
            # Outer boundary
            ([-6, -6], [ 6, -6]),
            ([ 6, -6], [ 6,  6]),
            ([ 6,  6], [-6,  6]),
            ([-6,  6], [-6, -6]),

            # Dead-end chamber
            ([-2, -6], [-2,  2]),
            ([-2,  2], [ 2,  2]),
            ([ 2,  2], [ 2, -2]),
            ([ 2, -2], [-4, -2]),
            ([-4, -2], [-4,  4]),
        ]
        start = [ 0.0, -5.0]
        goal  = [ 5.0,  5.0]

    elif name == "symmetric":
        walls = [
            # Outer boundary
            ([-6, -6], [ 6, -6]),
            ([ 6, -6], [ 6,  6]),
            ([ 6,  6], [-6,  6]),
            ([-6,  6], [-6, -6]),

            # Symmetric obstacles
            ([-2, -2], [-2,  2]),
            ([ 2, -2], [ 2,  2]),
            ([-2,  2], [ 2,  2]),
        ]
        start = [ 0.0, -5.0]
        goal  = [ 0.0,  5.0]

    else:
        raise ValueError(f"Unknown floor plan: {name}")

    # Convert to NumPy arrays
    walls = [(np.array(a, dtype=float), np.array(b, dtype=float)) for a, b in walls]
    start = np.array(start, dtype=float)
    goal  = np.array(goal, dtype=float)

    return walls, start, goal


In [ ]:
#@title Floor Plan Selection

# floor_plan = "baseline" #@param ["baseline", "zigzag", "deadend", "symmetric"]


# walls, start, goal = load_floor_plan(floor_plan)

# plot_floor_plan(walls, start, goal)


In [5]:
def quadratic_band_penalty(d, R):
    return 0.5 * (R - d)**2 if d <= R else 0.0

def quartic_band_penalty(d, R):
    return (R - d)**4 if d <= R else 0.0

def log_barrier_penalty(d, R, eps=1e-2):
    
    return np.log(R / (d + eps)) if d <= R else 0.0


In [6]:
def wall_cost(x, walls, R, w_wall, phi):
    total = 0.0
    for a, b in walls:
        d = point_to_segment_distance(x, a, b)
        total += w_wall * phi(d, R)
    return total

def goal_cost(x, goal, w_goal=1.0):
    # Simple smooth attraction (squared distance)
    return 0.5 * w_goal * np.sum((x - goal)**2)

def total_cost(x, walls, goal, R, w_wall, w_goal, phi):
    return goal_cost(x, goal, w_goal) + wall_cost(x, walls, R, w_wall, phi)


In [7]:
def grad_total_cost(x, walls, goal, R, w_wall, w_goal, phi, h=1e-3):
    # central differences
    ex = np.array([1.0, 0.0])
    ey = np.array([0.0, 1.0])

    fx1 = total_cost(x + h*ex, walls, goal, R, w_wall, w_goal, phi) 
    fx2 = total_cost(x - h*ex, walls, goal, R, w_wall, w_goal, phi)
    fy1 = total_cost(x + h*ey, walls, goal, R, w_wall, w_goal, phi)
    fy2 = total_cost(x - h*ey, walls, goal, R, w_wall, w_goal, phi)

    dfdx = (fx1 - fx2) / (2*h) # From slides 
    dfdy = (fy1 - fy2) / (2*h)
    return np.array([dfdx, dfdy])


In [8]:
def run_gradient_descent(start, walls, goal, R, w_wall, w_goal, phi,
                         alpha=0.05, max_iters=200000, tol=0.2):
    x = start.copy()
    traj = [x.copy()]

    for k in range(max_iters):
        g = grad_total_cost(x, walls, goal, R, w_wall, w_goal, phi)

        # gradient step
        x = x - alpha * g
        traj.append(x.copy())

        # stop if close to goal
        if np.linalg.norm(x - goal) <= tol:
            break

    return np.array(traj)


In [9]:
def plot_cost_and_trajectory(walls, start, goal, traj, R, w_wall, w_goal, phi,
                            xmin=-6, xmax=6, ymin=-6, ymax=6, N=180, step=8):
    xs = np.linspace(xmin, xmax, N)
    ys = np.linspace(ymin, ymax, N)
    X, Y = np.meshgrid(xs, ys)

    C = np.zeros_like(X)
    for i in range(N):
        for j in range(N):
            x = np.array([X[i, j], Y[i, j]])
            C[i, j] = total_cost(x, walls, goal, R, w_wall, w_goal, phi)

    dCy, dCx = np.gradient(C, ys, xs)

    plt.figure(figsize=(8, 7))
    plt.contourf(X, Y, C, levels=40, cmap="viridis")
    plt.colorbar(label="Total cost")

    # negative gradient field
    plt.quiver(X[::step, ::step], Y[::step, ::step],
               -dCx[::step, ::step], -dCy[::step, ::step],
               color="white", alpha=0.55)

    # walls
    for a, b in walls:
        plt.plot([a[0], b[0]], [a[1], b[1]], "k-", linewidth=3)

    # start/goal
    plt.scatter([start[0]], [start[1]], c="lime", s=80, edgecolors="k", label="Start")
    plt.scatter([goal[0]], [goal[1]], c="red",  s=80, edgecolors="k", label="Goal")

    # trajectory
    plt.plot(traj[:,0], traj[:,1], "r-", linewidth=2, label="Path")

    plt.axis("equal")
    plt.title("Cost field + gradient + Path")
    plt.legend()
    plt.show()


---

# Part 2: Multi-Particle Animation with Social Forces(used a lot functions of assignment 1)

This section extends the single-particle cost-function minimization framework to **N interacting particles**. Each particle moves according to gradient descent on its own cost function, which now includes contributions from:

1. **Goal attraction** — pulls each particle toward its target
2. **Wall repulsion** — penalizes proximity to walls  
3. **Social forces** — pairwise interaction penalties between particles

This is inspired by the Social Force Model for pedestrian dynamics (Helbing & Molnár, 1995), reinterpreted here as gradients of pairwise potential functions.

The cost function for particle $i$ is:

$$
C_i(\mathbf{x}_i) = C_{\text{goal}}(\mathbf{x}_i) + C_{\text{walls}}(\mathbf{x}_i) + \sum_{\substack{j=1 \\ j \neq i}}^{N} C_{\text{social}}(\mathbf{x}_i, \mathbf{x}_j)
$$

Each particle is updated independently via gradient descent:

$$
\mathbf{x}_i^{(k+1)} = \mathbf{x}_i^{(k)} - \alpha \nabla_{\mathbf{x}_i} C_i
$$

where $\alpha > 0$ is the step size.

---


## Task 1: Multi-Particle Simulation

### Setup and Infrastructure

Before introducing social forces, we first establish the multi-particle simulation infrastructure. This includes:
- Initializing $N$ particles with distinct starting positions
- Running simultaneous gradient-descent updates for all particles
- Visualizing collective trajectories

The social force term is set to zero in this task so we can observe how multiple particles behave under wall repulsion and goal attraction alone. This also serves as a **baseline** against which social force models will be compared.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle
from IPython.display import HTML


### Multi-Particle Cost and Gradient

We extend the single-particle total cost to include a social force term. For now the social penalty is a placeholder (returns 0) so Task 1 uses no inter-particle interaction.


In [ ]:
# ── Social force placeholder (no interaction) ─────────────────────────────────
def no_social(xi, xj, **kwargs):
    """Zero social force — baseline with no particle interaction."""
    return 0.0


# ── Per-particle total cost ────────────────────────────────────────────────────
def particle_cost(xi, positions, i, walls, goal_i,
                  R, w_wall, w_goal, w_social, phi_wall, phi_social, **social_kwargs):
    """
    Total cost for particle i:
        C_i = C_goal + C_walls + sum_{j != i} C_social(xi, xj)
    """
    c = goal_cost(xi, goal_i, w_goal)
    c += wall_cost(xi, walls, R, w_wall, phi_wall)
    for j, xj in enumerate(positions):
        if j != i:
            c += w_social * phi_social(xi, xj, **social_kwargs)
    return c


# ── Numerical gradient for particle i ─────────────────────────────────────────
def grad_particle_cost(xi, positions, i, walls, goal_i,
                       R, w_wall, w_goal, w_social, phi_wall, phi_social,
                       h=1e-3, **social_kwargs):
    """Central-difference gradient of particle i's cost w.r.t. its own position."""
    ex = np.array([1.0, 0.0])
    ey = np.array([0.0, 1.0])

    def c(x):
        return particle_cost(x, positions, i, walls, goal_i,
                             R, w_wall, w_goal, w_social,
                             phi_wall, phi_social, **social_kwargs)

    dfdx = (c(xi + h*ex) - c(xi - h*ex)) / (2*h)
    dfdy = (c(xi + h*ey) - c(xi - h*ey)) / (2*h)
    return np.array([dfdx, dfdy])





Multi-particle cost and gradient functions defined.


In [ ]:
# ── Multi-particle gradient descent ───────────────────────────────────────────
def run_multi_particle(starts, walls, goals,
                       R, w_wall, w_goal, w_social,
                       phi_wall, phi_social,
                       alpha=0.01, max_iters=3000, tol=0.05,
                       **social_kwargs):
    """
    Simulate N particles simultaneously via gradient descent.

    Parameters
    ----------
    starts  : list of np.array, shape (N, 2) — initial positions
    goals   : list of np.array or single np.array — goal(s)
    Returns
    -------
    trajs : np.array, shape (steps, N, 2)
    """
    N = len(starts)
    positions = [np.array(s, dtype=float) for s in starts]

    # If a single goal is given, broadcast to all particles
    if isinstance(goals, np.ndarray) and goals.ndim == 1:
        goals = [goals] * N
    else:
        goals = [np.array(g, dtype=float) for g in goals]

    trajs = [positions.copy()]

    for k in range(max_iters):
        grads = []
        for i in range(N):
            g = grad_particle_cost(
                positions[i], positions, i,
                walls, goals[i],
                R, w_wall, w_goal, w_social,
                phi_wall, phi_social, **social_kwargs
            )
            grads.append(g)

        # Update all particles simultaneously
        new_positions = [positions[i] - alpha * grads[i] for i in range(N)]
        positions = new_positions
        trajs.append(positions.copy())

        # Stop when all particles reach their goal
        if all(np.linalg.norm(positions[i] - goals[i]) <= tol for i in range(N)):
            print(f"  All particles reached goal at iteration {k+1}.")
            break

    return np.array(trajs)   # shape: (steps, N, 2)





### Visualization

We define two visualization helpers:
1. A static plot showing all trajectories overlaid on the floor plan and cost field
2. An animated version showing particles moving in real time


In [ ]:
def plot_multi_trajectories(walls, starts, goals, trajs,
                            title="Multi-Particle Trajectories",
                            xmin=-6, xmax=6, ymin=-6, ymax=6):
    
    N = trajs.shape[1]
    colors = plt.cm.tab10(np.linspace(0, 1, N))

    fig, ax = plt.subplots(figsize=(7, 7))

    # Floor plan walls
    for a, b in walls:
        ax.plot([a[0], b[0]], [a[1], b[1]], 'k-', linewidth=2.5)

    for i in range(N):
        c = colors[i]
        traj_i = trajs[:, i, :]
        ax.plot(traj_i[:, 0], traj_i[:, 1], '-', color=c, linewidth=1.5, alpha=0.8)   #Plot all particle trajectories on top of the floor plan.
   
    
        ax.scatter(*starts[i], color=c, s=80, edgecolors='k', zorder=5,
                   marker='o', label=f'P{i+1} start')
        ax.scatter(*goals[i] if hasattr(goals[i], '__len__') else goals,     
                   color=c, s=100, edgecolors='k', zorder=5, marker='*')         #Each particle gets a distinct colour.

    ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)
    ax.set_aspect('equal'); ax.grid(True, alpha=0.3)
    ax.set_title(title, fontsize=13)
    ax.legend(fontsize=8, loc='upper left')
    plt.tight_layout()
    plt.show()


def animate_particles(walls, starts, goals, trajs,
                      title="Multi-Particle Animation",
                      xmin=-6, xmax=6, ymin=-6, ymax=6, interval=30):    #Return an HTML animation of particles moving along their trajectories.
    
   
    
    N = trajs.shape[1]
    T = trajs.shape[0]
    colors = plt.cm.tab10(np.linspace(0, 1, N))

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)
    ax.set_aspect('equal'); ax.grid(True, alpha=0.3)
    ax.set_title(title, fontsize=12)

    for a, b in walls:
        ax.plot([a[0], b[0]], [a[1], b[1]], 'k-', linewidth=2.5)

    # Goals
    for i, g in enumerate(goals if hasattr(goals[0], '__len__') else [goals]*N):
        ax.scatter(*g, color=colors[i], s=120, marker='*', edgecolors='k', zorder=4)

    # Particle dots and trail lines
    dots  = [ax.plot([], [], 'o', color=colors[i], ms=8, zorder=5)[0] for i in range(N)]
    trails = [ax.plot([], [], '-', color=colors[i], lw=1.2, alpha=0.5)[0] for i in range(N)]

    step = max(1, T // 300)   # sub-sample for smoother animation

    def init():
        for d, tr in zip(dots, trails):
            d.set_data([], [])
            tr.set_data([], [])
        return dots + trails

    def update(frame):
        t = frame * step
        t = min(t, T - 1)
        for i in range(N):
            x, y = trajs[t, i]
            dots[i].set_data([x], [y])
            t0 = max(0, t - 60)
            trails[i].set_data(trajs[t0:t+1, i, 0], trajs[t0:t+1, i, 1])
        return dots + trails

    n_frames = T // step
    ani = animation.FuncAnimation(fig, update, frames=n_frames,
                                  init_func=init, blit=True, interval=interval)
    plt.close(fig)
    return HTML(ani.to_jshtml())




### Experiment 1.1 — Baseline Floor Plan, No Social Forces

We run **5 particles** on the **baseline** (L-shaped) floor plan, all heading to the same goal. No inter-particle interaction is active. This establishes a reference for later comparison once social forces are introduced.

**Setup:**
- Floor plan: *baseline*
- Penalty: Quadratic Band, $R = 1.0$, $w_{\text{wall}} = 30$, $w_{\text{goal}} = 5$
- No social force ($w_{\text{social}} = 0$)
- Particles are initialized at distinct positions near the start region


In [14]:
# ── Load floor plan ───────────────────────────────────────────────────────────
walls, start_ref, goal_ref = load_floor_plan("baseline")

# ── 5 particles starting at different positions ───────────────────────────────
N = 5
np.random.seed(42)
# Scatter starts near the reference start point
starts = [(-2,1.5),(-3,-1.5),(-4,-4),(-4,1),(-3,1)]
goals  = [goal_ref] * N   # shared goal

print("Starting positions:")
for i, s in enumerate(starts):
    print(f"  P{i+1}: {s}")
print(f"\nCommon goal: {goal_ref}")

# ── Hyperparameters ───────────────────────────────────────────────────────────
R       = 1.0
w_wall  = 30.0
w_goal  = 5.0
w_social = 0.0    # No social forces for Task 1 baseline
alpha   = 0.001


Starting positions:
  P1: (-2, 1.5)
  P2: (-3, -1.5)
  P3: (-4, -4)
  P4: (-4, 1)
  P5: (-3, 1)

Common goal: [4. 4.]


In [ ]:
# ── Run simulation ────────────────────────────────────────────────────────────
print("Running multi-particle simulation (no social forces)...")
trajs_baseline = run_multi_particle(
    starts, walls, goals,
    R=R, w_wall=w_wall, w_goal=w_goal, w_social=w_social,
    phi_wall=quadratic_band_penalty,
    phi_social=no_social,
    alpha=alpha, max_iters=50000, tol=0.05
)
print(f"Trajectory shape: {trajs_baseline.shape}  (steps × particles × 2)")


Running multi-particle simulation (no social forces)...
  All particles reached goal at iteration 1118.
Trajectory shape: (1119, 5, 2)  (steps × particles × 2)


In [ ]:
# ── Visualize ─────────────────────────────────────────────────────────────────
plot_multi_trajectories(
    walls, starts, goals, trajs_baseline,
    title="Task 1 — 5 Particles, No Social Forces (Baseline Floor Plan)"
)


In [ ]:
# ── Animation ─────────────────────────────────────────────────────────────────
animate_particles(
    walls, starts, goals, trajs_baseline,
    title="Task 1 — 5 Particles, No Social Forces (Baseline)"
)


### Experiment 1.2 — Zigzag Floor Plan, No Social Forces

We repeat the experiment on the **zigzag** floor plan, which is a narrow zig-zag corridor. With no social force, each particle navigates independently. The key observation here is whether all particles can navigate the tight turns, and whether the trajectories overlap or spread naturally.


In [ ]:
# ── Load zigzag floor plan ────────────────────────────────────────────────────
walls_zz, start_zz, goal_zz = load_floor_plan("zigzag")

N = 5
starts_zz = [(-3,-3.5),(-5,-1),start_zz,(-1.8,-3),(-4,2) ]
goals_zz  = [goal_zz] * N

print("Running zigzag simulation (no social forces)...")
trajs_zz = run_multi_particle(
    starts_zz, walls_zz, goals_zz,
    R=R, w_wall=w_wall, w_goal=w_goal, w_social=0.0,
    phi_wall=quadratic_band_penalty,
    phi_social=no_social,
    alpha=alpha, max_iters=50000, tol=0.05
)

plot_multi_trajectories(
    walls_zz, starts_zz, goals_zz, trajs_zz,
    title="Task 1 — 5 Particles, No Social Forces (Zigzag Floor Plan)",
    xmin=-7, xmax=7, ymin=-5, ymax=5
)


### Observations — Task 1

Without social forces, the particles behave as **N independent single-particle agents**. Each follows the gradient of its own wall-repulsion + goal-attraction cost, completely ignoring the other particles. Key observations:

| Observation | Detail |
|---|---|
| **Path independence** | Each particle traces the same path, offset only by its starting position |
| **No collision avoidance** | Particles can pass through each other — no inter-particle repulsion |
| **Wall avoidance preserved** | All single-particle wall penalties work correctly for each agent |
| **Convergence** | All particles reach the goal as expected |

This confirms the infrastructure is working correctly and establishes the **no-interaction baseline** for comparison in Tasks 2–4.

> **Note:** The identical trajectories (up to start offset) are expected — and undesirable in realistic multi-agent scenarios. This motivates the introduction of social force terms in the next task.


---

## Task 2: Social Force Models

### Overview

In Task 1, particles moved independently with no awareness of each other. In reality, agents — whether pedestrians, robots, or virtual characters — must **avoid collisions** and exhibit **realistic group behavior**. This is achieved by adding a **social force penalty term** $C_{\text{social}}(\mathbf{x}_i, \mathbf{x}_j)$ to each particle's cost function.

We implement and compare **three isotropic social force models**, all based on the pairwise distance:

$$
d_{ij} = \|\mathbf{x}_i - \mathbf{x}_j\|
$$

The three models are:

| Model | Formula | Character |
|---|---|---|
| **Quadratic Repulsion** | $\frac{1}{2}(R - d_{ij})^2$ if $d_{ij} \leq R$ | Soft personal space |
| **Inverse-Distance** | $\frac{1}{(d_{ij} + \varepsilon)^p}$ | Strong short-range repulsion |
| **Exponential** | $A \exp(-d_{ij}/B)$ | Long-range smooth interaction |

All three models produce repulsion — they increase in cost as particles get closer — but they differ in range, gradient sharpness, and sensitivity to parameters.


### 2.1 Implementing the Social Force Models


In [ ]:


# ── 1: Quadratic Repulsion (Personal Space) ─────────────────────────────
def social_quadratic(xi, xj, R_s=1.5, **kwargs):
   
    d = np.linalg.norm(xi - xj)
    if d <= R_s:
        return 0.5 * (R_s - d) ** 2
    return 0.0


# ── 2: Inverse-Distance Repulsion ───────────────────────────────────────
def social_inverse(xi, xj, eps=1e-2, p=2, R_s=2.0, **kwargs):
   
    d = np.linalg.norm(xi - xj)
    if d <= R_s:
        return 1.0 / (d + eps) ** p
    return 0.0


# ── 3: Exponential Social Force ─────────────────────────────────────────
def social_exponential(xi, xj, A=1.0, B=0.5, **kwargs):
   
    d = np.linalg.norm(xi - xj)
    return A * np.exp(-d / B)





### 2.1b  Penalty Function Profiles — 1D Curves and 2D Fields

Before running multi-particle simulations, it is important to understand the **shape of each social penalty function** in isolation. Inspired by the reference notes style, we plot each function in two complementary ways:

- **Left:** 1D profile of $\varphi(d)$ vs. distance $d$ — shows how the penalty grows as two particles approach each other, and where it vanishes
- **Right:** 2D radial field — the penalty evaluated at every point $(x, y)$ in the plane, with the source particle at the origin, visualizing the "influence zone"

These plots make it easy to compare range, sharpness, and the effect of key parameters ($R_s$, $\varepsilon$, $p$, $B$).


In [3]:


def plot_penalty_profile(phi_fn, param_sets, param_label,
                         d_max=3.5, title="", color_cycle=None):
    """
    Left : 1D curve phi(d) vs d for each parameter set.
    Right: 2D radial heatmap for the first (default) parameter set.
    Annotates key values (R, truncation point) on the plots.
    """
    if color_cycle is None:
        color_cycle = ['#1f77b4','#ff7f0e','#2ca02c','#d62728']

    fig, axes = plt.subplots(1, 2, figsize=(13, 4.5))
    ax1, ax2 = axes

    # ── Left: 1D profiles ────────────────────────────────────────────────────
    d_vals = np.linspace(1e-3, d_max, 500)

    for idx, (kw, label) in enumerate(param_sets):
        phi_vals = np.array([phi_fn(
            np.array([d, 0.0]), np.array([0.0, 0.0]), **kw
        ) for d in d_vals])
        ax1.plot(d_vals, phi_vals, color=color_cycle[idx % len(color_cycle)],
                 lw=2, label=label)

        # Annotate truncation / influence radius R_s if present
        R_s = kw.get('R_s', None)
        if R_s is not None and idx == 0:
            ax1.axvline(R_s, color=color_cycle[idx], lw=1.2, ls='--', alpha=0.6)
            ax1.text(R_s + 0.05, ax1.get_ylim()[1]*0.05 if ax1.get_ylim()[1]>0 else 0.1,
                     f'$R_s={R_s}$', color=color_cycle[idx], fontsize=9)

    ax1.axhline(0, color='gray', lw=0.8, ls=':')
    ax1.set_xlabel('Distance $d$', fontsize=11)
    ax1.set_ylabel('$\\varphi(d)$', fontsize=11)
    ax1.set_title(f'{title}\n1D Penalty Profile', fontsize=11)
    ax1.legend(fontsize=9)
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, d_max)
    ax1.set_ylim(bottom=0)

    # ── Right: 2D radial field (use first param set) ──────────────────────────
    kw0 = param_sets[0][0]
    N_grid = 120
    lim = d_max * 0.9
    xs = np.linspace(-lim, lim, N_grid)
    ys = np.linspace(-lim, lim, N_grid)
    X, Y = np.meshgrid(xs, ys)
    source = np.array([0.0, 0.0])

    Z = np.zeros_like(X)
    for i in range(N_grid):
        for j in range(N_grid):
            Z[i,j] = phi_fn(np.array([X[i,j], Y[i,j]]), source, **kw0)

    im = ax2.imshow(Z, origin='lower', extent=[-lim, lim, -lim, lim],
                    cmap='plasma', aspect='equal', interpolation='bilinear')
    plt.colorbar(im, ax=ax2, label='$\\varphi(d)$')

    # Draw influence radius circle
    R_s = kw0.get('R_s', None)
    if R_s is not None:
        theta = np.linspace(0, 2*np.pi, 200)
        ax2.plot(R_s*np.cos(theta), R_s*np.sin(theta),
                 'w--', lw=1.5, label=f'$R_s = {R_s}$')
        ax2.legend(fontsize=9, loc='upper right')

    ax2.scatter(0, 0, c='red', s=130, zorder=5, label='Source')
    ax2.set_title(f'{title}\n2D Radial Field ({param_sets[0][1]})', fontsize=11)
    ax2.set_xlabel('$x$', fontsize=11); ax2.set_ylabel('$y$', fontsize=11)

    plt.tight_layout()
    plt.show()





In [ ]:


from mpl_toolkits.mplot3d import Axes3D

def plot_1d_and_3d_surface(phi_fn, kw, name, d_max=3.0, color='#1f77b4'):
   
    N_g = 100
    lim = d_max
    xs = np.linspace(-lim, lim, N_g)
    ys = np.linspace(-lim, lim, N_g)
    X, Y = np.meshgrid(xs, ys)
    source = np.array([0.0, 0.0])

    Z = np.zeros_like(X)
    for i in range(N_g):
        for j in range(N_g):
            Z[i, j] = phi_fn(np.array([X[i,j], Y[i,j]]), source, **kw)

    d_vals = np.linspace(1e-3, d_max, 500)
    phi_1d = [phi_fn(np.array([d, 0.0]), source, **kw) for d in d_vals]

    fig = plt.figure(figsize=(15, 5))

    # ── Left: 1D profile ──────────────────────────────────────────────────────
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(d_vals, phi_1d, lw=2.5, color=color)
    ax1.fill_between(d_vals, phi_1d, alpha=0.15, color=color)
    R_s = kw.get('R_s', None)
    if R_s:
        ax1.axvline(R_s, color='gray', lw=1.2, ls='--')
        ax1.text(R_s + 0.05, max(phi_1d)*0.05, f'$R_s={R_s}$', color='gray', fontsize=9)
    ax1.set_xlabel('distance (d)', fontsize=11)
    ax1.set_ylabel('F(d)', fontsize=11)
    ax1.set_title(f'{name}\n', fontsize=10)
    ax1.set_xlim(0, d_max); ax1.set_ylim(bottom=0)
    ax1.grid(True, alpha=0.3)

    # ── Right: 3D surface ─────────────────────────────────────────────────────
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    ax2.plot_surface(X, Y, Z, cmap='plasma', linewidth=0, antialiased=True, alpha=0.92)
    ax2.set_xlabel('x', fontsize=9); ax2.set_ylabel('y', fontsize=9)
    ax2.set_zlabel('F(d)', fontsize=9)
    ax2.set_title(f'{name}\n', fontsize=10)
    ax2.view_init(elev=30, azim=-60)

    plt.suptitle(f'Social Penalty: {name}', fontsize=12, fontweight='bold', y=1.01)
    plt.tight_layout()
    plt.show()


# ── Plot all three isotropic models ──────────────────────────────────────────
plot_1d_and_3d_surface(social_quadratic,   {'R_s': 1.5},           'Quadratic Repulsion ($R_s=1.5$)',    color='#1f77b4')
plot_1d_and_3d_surface(social_inverse,     {'p': 1, 'eps': 0.05, 'R_s': 1.0}, 'Inverse-Distance ($p=1, eps=0.05$)', color='#ff7f0e')
plot_1d_and_3d_surface(social_exponential, {'A': 1.0, 'B': 0.5},  'Exponential ($A=1, B=0.5$)',         color='#2ca02c')


#### Quadratic Repulsion — Personal Space Penalty

$$
\varphi(d) = \begin{cases} \tfrac{1}{2}(R_s - d)^2 & d \leq R_s \\ 0 & d > R_s \end{cases}
$$

The quadratic penalty creates a smooth "buffer zone" around each particle. The penalty is zero at $d = R_s$ and rises parabolically toward $d = 0$. The dashed line marks $R_s$ — the boundary of the personal space bubble. Varying $R_s$ controls both the range and the maximum penalty height.


#### Inverse-Distance Repulsion

$$
\varphi(d) = \begin{cases} \dfrac{1}{(d + \varepsilon)^p} & d \leq R_s \\ 0 & d > R_s \end{cases}
$$

The inverse-distance penalty is strongly singular near $d = 0$ (controlled by $\varepsilon$) and decays as a power law. The exponent $p$ controls how rapidly the penalty falls off: higher $p$ concentrates repulsion at very short range. The 2D field shows a bright "hot core" around the source particle, sharply contained within $R_s$.


The singularity zoom above illustrates why $\varepsilon$ is a critical parameter for the inverse-distance model. Too small an $\varepsilon$ (e.g. $10^{-3}$) causes the gradient to spike to enormous values near contact, potentially destabilizing gradient descent. A value of $\varepsilon \approx 0.01$–$0.1$ keeps the penalty large but numerically tractable.


#### Exponential Social Force

$$
\varphi(d) = A \exp\!\left(-\dfrac{d}{B}\right)
$$

The exponential model has **infinite support** — it never drops to exactly zero, but decays rapidly. The decay length $B$ controls how far the influence extends: larger $B$ produces a wider, flatter field. Unlike the truncated models, there is no hard influence radius; the 2D field shows a smooth, continuous "aura" around the source particle.


#### All Three Forces Comparision
The plot  overlays all three penalty functions on the same axes, with parameters chosen so that each model produces roughly equal penalty at $d = 0.05$. This reveals the fundamental differences in **shape**:

- Quadratic: zero outside $R_s$, smooth parabolic rise inside
- Inverse-distance: power-law spike near contact, truncated at $R_s$  
- Exponential: smooth continuous decay, no hard cutoff


### 2.2 Visualizing the Social Force Fields

Before running multi-particle simulations, it is instructive to visualize the **cost field and gradient** induced by a single stationary particle at the origin. This reveals the shape and range of each model's repulsive field.

We place a **source particle** at the origin and compute the social cost at every point in the grid, then plot the negative gradient (the direction a nearby particle would be pushed).


**Observations from the force field plots:**

- **Quadratic Repulsion** has a compact, bounded influence zone. The repulsion is zero beyond $R_s$ and grows smoothly toward the source particle. The gradient magnitude is moderate, making it the most stable for gradient descent.

- **Inverse-Distance** produces a very intense repulsion close to the source and decays rapidly. The cost spikes near the origin, which can cause large gradient steps and numerical stiffness if $\varepsilon$ is too small.

- **Exponential** creates a smooth, radially symmetric field with infinite support (never exactly zero). The gradient is well-behaved everywhere, but the interaction range is controlled entirely by the parameter $B$.

All three fields are **isotropic** — the cost depends only on distance, not direction, so the gradient field is perfectly radially symmetric around the source particle.


### 2.3 Multi-Particle Simulation — Comparing Social Force Models

We now run the multi-particle simulation on the **baseline** floor plan with each social force model. Parameters are kept identical across models to enable a fair comparison.

**Experimental setup:**
- Floor plan: *baseline* (L-shaped corridor)
- $N = 5$ particles, same starts and shared goal as Task 1
- Wall penalty: Quadratic Band, $R = 1.0$, $w_{\text{wall}} = 30$, $w_{\text{goal}} = 2$
- Social weight: $w_{\text{social}} = 5$
- Step size: $\alpha = 0.001$


In [ ]:
# ── Shared setup ──────────────────────────────────────────────────────────────
walls, start_ref, goal_ref = load_floor_plan("baseline")
N = 5
np.random.seed(2026)
starts = [(-2,1.5),(-3,-1.5),(-4,-4),(-4,1),(-3,1)]
goals  = [goal_ref] * N

R       = 1.0
w_wall  = 30.0
w_goal  = 2.0
w_social =5.0
alpha   = 0.001

# ── Run each model ────────────────────────────────────────────────────────────
experiments = [
    ("No Social Force",      no_social,          {}),
    ("Quadratic Repulsion",  social_quadratic,   {'R_s': 1.5}),
    ("Inverse-Distance",     social_inverse,     {'p': 2, 'R_s': 2.0, 'eps': 1e-2}),
    ("Exponential",          social_exponential, {'A': 1.0, 'B': 0.5}),
]

results = {}
for name, phi_s, kw in experiments:
    print(f"Running: {name}...")
    traj = run_multi_particle(
        starts, walls, goals,
        R=R, w_wall=w_wall, w_goal=w_goal, w_social=w_social,
        phi_wall=quadratic_band_penalty,
        phi_social=phi_s,
        alpha=alpha, max_iters=50000, tol=0.05,
        **kw
    )
    results[name] = traj
    print(f"  -> {traj.shape[0]} steps")


In [ ]:
# ── Side-by-side trajectory plots ─────────────────────────────────────────────
fig, axes = plt.subplots(1, 4, figsize=(22, 6))
colors = plt.cm.tab10(np.linspace(0, 1, N))

for ax, (name, traj) in zip(axes, results.items()):
    for a, b in walls:
        ax.plot([a[0],b[0]], [a[1],b[1]], 'k-', linewidth=2.5)
    for i in range(N):
        ax.plot(traj[:,i,0], traj[:,i,1], '-', color=colors[i],
                linewidth=1.5, alpha=0.85)
        ax.scatter(*starts[i], color=colors[i], s=60, edgecolors='k', zorder=5)
        ax.scatter(*goal_ref, color=colors[i], s=100, marker='*', edgecolors='k', zorder=5)
    ax.set_xlim(-6,6); ax.set_ylim(-6,6)
    ax.set_aspect('equal'); ax.grid(True, alpha=0.25)
    steps = results[name].shape[0]
    ax.set_title(f"{name}\n({steps} steps)", fontsize=10)
    ax.set_xlabel('x'); ax.set_ylabel('y')

plt.suptitle("Task 2 — Comparison of Social Force Models (Baseline Floor Plan, N=5)",
             fontsize=13, y=1.01)
plt.tight_layout()
plt.show()


### 2.4 Quantitative Comparison

To compare the models more rigorously, we measure three quantities for each simulation:

1. **Steps to convergence** — total gradient descent iterations until all particles reach the goal
2. **Mean minimum inter-particle distance** — average of the minimum pairwise distance recorded during the run (higher = better separation)
3. **Path length** — total Euclidean path length traveled by each particle (averaged across particles)


In [30]:
import itertools

def mean_min_interparticle_distance(traj):
    """Average over time of the minimum pairwise distance between any two particles."""
    T, N, _ = traj.shape
    min_dists = []
    for t in range(T):
        positions = traj[t]  # shape (N, 2)
        dists = [np.linalg.norm(positions[i] - positions[j])
                 for i,j in itertools.combinations(range(N), 2)]
        min_dists.append(min(dists))
    return np.mean(min_dists)

def mean_path_length(traj):
    """Average path length across all particles."""
    lengths = []
    for i in range(traj.shape[1]):
        path = traj[:, i, :]
        diffs = np.diff(path, axis=0)
        lengths.append(np.sum(np.linalg.norm(diffs, axis=1)))
    return np.mean(lengths)

print(f"{'Model':<25} {'Steps':>8} {'Mean Min Dist':>15} {'Avg Path Length':>17}")
print("-" * 70)
for name, traj in results.items():
    steps  = traj.shape[0]
    mmid   = mean_min_interparticle_distance(traj)
    mpl    = mean_path_length(traj)
    print(f"{name:<25} {steps:>8}  {mmid:>14.3f}  {mpl:>16.3f}")


Model                        Steps   Mean Min Dist   Avg Path Length
----------------------------------------------------------------------
No Social Force               2456           0.138             9.564
Quadratic Repulsion          15001           0.737             9.391
Inverse-Distance             15001           1.242            40.838
Exponential                  15001           0.673             9.447


### 2.5 Analysis and Discussion

**Effect of social forces on collective behavior:**

The simulation results reveal that even simple pairwise penalty terms have a significant impact on how particles navigate together. Without social forces, all particles converge on nearly identical paths — they have no reason to separate. Once social forces are introduced, particles spread out to maintain personal space, producing qualitatively different collective trajectories.

**Quadratic Repulsion** enforces a soft "bubble" around each particle. Within radius $R_s$, the penalty grows smoothly, gently pushing particles apart. The resulting paths show clear lateral separation while remaining smooth. This model converges reliably and is the most numerically stable of the three.

**Inverse-Distance Repulsion** applies very strong short-range forces. Particles that come close are strongly deflected, which can cause sharp turns and oscillations near narrow corridors where wall repulsion and social repulsion compete. The convergence can be slower because large gradient magnitudes near other particles require a small step size.

**Exponential Social Force**, inspired by Helbing & Molnár (1995), produces long-range but soft interactions. Because the cost never drops to exactly zero, every particle is always slightly aware of all others. This encourages a smoother, more spread-out formation but the effect at distance is subtle unless the weight $w_{\text{social}}$ or parameter $A$ is large.

**My opinion:** The choice of social force model affects not just path shape but also numerical stability and convergence speed. The Quadratic Repulsion model offers the best balance of physical realism, smoothness, and ease of tuning — consistent with the assignment's recommendation to start with quadratic penalties before moving to more aggressive formulations.


### 2.6 Zigzag Floor Plan — Social Forces Under Corridor Constraints

Narrow corridors are where social forces are most challenging. We repeat the experiment on the zigzag floor plan to observe how inter-particle repulsion interacts with wall repulsion in tight spaces.


In [ ]:
walls_zz, start_zz, goal_zz = load_floor_plan("zigzag")
N = 5
starts_zz = [(-3,-3.5),(-5,-1),start_zz,(-1.8,-3),(-4,2) ]
goals_zz  = [goal_zz] * N

fig, axes = plt.subplots(1, 3, figsize=(17, 6))
social_models = [
    ("Quadratic Repulsion",  social_quadratic,   {'R_s': 1.0}),
    ("Inverse-Distance",     social_inverse,     {'p': 2, 'R_s': 1.5, 'eps': 1e-2}),
    ("Exponential",          social_exponential, {'A': 1.0, 'B': 0.4}),
]
colors = plt.cm.tab10(np.linspace(0, 1, N))

for ax, (name, phi_s, kw) in zip(axes, social_models):
    print(f"Running zigzag: {name}...")
    traj = run_multi_particle(
        starts_zz, walls_zz, goals_zz,
        R=1.0, w_wall=30.0, w_goal=2.0, w_social=5.0,
        phi_wall=quadratic_band_penalty,
        phi_social=phi_s,
        alpha=0.001, max_iters=50000, tol=0.05,
        **kw
    )
    for a, b in walls_zz:
        ax.plot([a[0],b[0]], [a[1],b[1]], 'k-', linewidth=2.5)
    for i in range(N):
        ax.plot(traj[:,i,0], traj[:,i,1], '-', color=colors[i], lw=1.5, alpha=0.85)
        ax.scatter(*starts_zz[i], color=colors[i], s=60, edgecolors='k', zorder=5)
        ax.scatter(*goal_zz, color=colors[i], s=100, marker='*', edgecolors='k', zorder=5)
    ax.set_xlim(-7,7); ax.set_ylim(-5,5)
    ax.set_aspect('equal'); ax.grid(True, alpha=0.25)
    ax.set_title(f"{name}\n({traj.shape[0]} steps)", fontsize=10)

plt.suptitle("Task 2 — Social Forces in Zigzag Corridor (N=5)", fontsize=13, y=1.01)
plt.tight_layout()
plt.show()



**Observations in narrow corridors:**

In the zigzag floor plan, the interaction between wall repulsion and social repulsion becomes more apparent. When particles are forced into single-file by the corridor geometry, the social force models exhibit different behaviors:

- **Quadratic Repulsion** — particles naturally form a queue, maintaining spacing equal to roughly $R_s$. The trailing particles are gently pushed forward as the leading particle moves ahead.
- **Inverse-Distance** — the strong near-contact penalty can cause the particle queue to "accordion" — particles alternately bunch up and spread out as they navigate turns.
- **Exponential** — the long-range soft repulsion tends to spread particles along the corridor in a more uniform way, but the effect diminishes in very tight bends where wall forces dominate.

These results suggest that in constrained environments, the **range** of the social force (controlled by $R_s$ or $B$) is as important as its magnitude. A social force radius larger than the corridor width can create local minima where particles are trapped between wall and social repulsion.


---

## Task 3: Visualization and Animation

### Overview

Good visualization is essential for understanding multi-particle dynamics. Static trajectory plots reveal final path shapes, but they cannot capture **timing, congestion, and emergent ordering** — behaviors that only become visible when watching the simulation unfold over time.

In this task we produce three complementary visualizations:

1. **Particle trail animation** — real-time animation with fading trails showing recent history
2. **Congestion heatmap** — a spatial density map showing where particles spent the most time (and therefore where crowding occurred)
3. **Inter-particle distance over time** — time-series plot tracking separation between all particle pairs, exposing oscillations, near-misses, and deadlocks

These three views together give a complete picture of how each social force model shapes collective motion.


### 3.1 Enhanced Animation with Velocity Indicators

We extend the basic `animate_particles` function with **velocity arrows** on each particle, making it possible to see speed and direction of travel at each frame. We also add a **real-time step counter** and color the particles by their distance to the goal.


In [ ]:
def animate_particles_enhanced(walls, starts, goals_list, trajs,
                                title="Multi-Particle Animation",
                                xmin=-6, xmax=6, ymin=-6, ymax=6,
                                interval=25, trail_len=80):
    """
    Enhanced animation:
      - Fading trajectory trails
      - Velocity arrows on each particle
      - Step counter
      - Goal markers
    Returns an HTML animation object.
    """
    N    = trajs.shape[1]
    T    = trajs.shape[0]
    step = max(1, T // 600)        # subsample to ~400 frames max
    colors = plt.cm.tab10(np.linspace(0, 1, N))

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)
    ax.set_aspect('equal'); ax.grid(True, alpha=0.25)
    ax.set_title(title, fontsize=11)

    # Walls
    for a, b in walls:
        ax.plot([a[0],b[0]], [a[1],b[1]], 'k-', linewidth=2.5)

    # Goal markers
    for i, g in enumerate(goals_list):
        ax.scatter(*g, color=colors[i], s=140, marker='*',
                   edgecolors='k', linewidths=0.8, zorder=4)

    # Start markers (static)
    for i, s in enumerate(starts):
        ax.scatter(*s, color=colors[i], s=60, marker='o',
                   edgecolors='k', linewidths=0.8, zorder=4, alpha=0.4)

    # Animated elements
    dots   = [ax.plot([], [], 'o', color=colors[i], ms=9,
                      markeredgecolor='k', markeredgewidth=0.6, zorder=6)[0]
              for i in range(N)]
    trails = [ax.plot([], [], '-', color=colors[i], lw=1.4, alpha=0.45)[0]
              for i in range(N)]
    arrows = [ax.annotate('', xy=(0,0), xytext=(0,0),
                          arrowprops=dict(arrowstyle='->', color=colors[i],
                                         lw=1.5))
              for i in range(N)]
    timer  = ax.text(0.02, 0.97, '', transform=ax.transAxes,
                     fontsize=9, va='top',
                     bbox=dict(boxstyle='round', fc='white', alpha=0.7))

    def init():
        for d, tr in zip(dots, trails):
            d.set_data([], [])
            tr.set_data([], [])
        timer.set_text('')
        return dots + trails + [timer]

    def update(frame):
        t = min(frame * step, T - 1)
        for i in range(N):
            x, y = trajs[t, i]
            dots[i].set_data([x], [y])

            # Trail
            t0 = max(0, t - trail_len)
            trails[i].set_data(trajs[t0:t+1, i, 0], trajs[t0:t+1, i, 1])

            # Velocity arrow (finite diff)
            if t > 0:
                vx = trajs[t, i, 0] - trajs[t-1, i, 0]
                vy = trajs[t, i, 1] - trajs[t-1, i, 1]
                spd = np.hypot(vx, vy)
                if spd > 1e-6:
                    scale = 0.4
                    arrows[i].set_position((x, y))
                    arrows[i].xy = (x + vx/spd*scale, y + vy/spd*scale)
                    arrows[i].xyann = (x, y)

        timer.set_text(f'step {t}')
        return dots + trails + arrows + [timer]

    n_frames = T // step
    ani = animation.FuncAnimation(fig, update, frames=n_frames,
                                  init_func=init, blit=False, interval=interval)
    plt.close(fig)
    return HTML(ani.to_jshtml())





### 3.2 Congestion Heatmap

The **congestion heatmap** accumulates a 2D density of particle positions over all time steps. Regions where particles lingered — near walls, at corridor turns, or in crowded areas — appear as hot spots. This is analogous to a pedestrian flow density map.


In [ ]:
def plot_congestion_heatmap(walls, trajs, title="Congestion Heatmap",
                            xmin=-6, xmax=6, ymin=-6, ymax=6,
                            bins=80, ax=None):
   
    standalone = ax is None
    if standalone:
        fig, ax = plt.subplots(figsize=(6, 6))

    # Flatten all particle positions: shape (T*N, 2)
    all_pos = trajs.reshape(-1, 2)

    h, xedges, yedges = np.histogram2d(
        all_pos[:,0], all_pos[:,1],
        bins=bins, range=[[xmin,xmax],[ymin,ymax]]
    )
    # Gaussian smooth for cleaner appearance
    from scipy.ndimage import gaussian_filter
    h_smooth = gaussian_filter(h.T, sigma=1.5)

    im = ax.imshow(h_smooth, origin='lower', aspect='equal',
                   extent=[xmin, xmax, ymin, ymax],
                   cmap='hot', interpolation='bilinear')
    if standalone:
        plt.colorbar(im, ax=ax, label='Accumulated presence')

    for a, b in walls:
        ax.plot([a[0],b[0]], [a[1],b[1]], 'w-', linewidth=2.5)

    ax.set_title(title, fontsize=11)
    ax.set_xlabel('x'); ax.set_ylabel('y')

    if standalone:
        plt.tight_layout()
        plt.show()
    return ax




### 3.3 Inter-Particle Distance Over Time

Tracking **pairwise distances over time** reveals:
- Whether particles maintain safe separation throughout the run
- Oscillatory behavior (distance cycling up and down)
- Near-miss events (distance approaching zero)
- Whether the social force successfully keeps particles apart


In [ ]:
def plot_interparticle_distances(trajs, title="Inter-Particle Distances Over Time", ax=None):
    """
    Plot all pairwise distances d_ij(t) over the simulation.
    Also plots mean and minimum envelope.
    """
    T, N, _ = trajs.shape
    pairs = list(itertools.combinations(range(N), 2))
    dist_matrix = np.zeros((T, len(pairs)))

    for k, (i, j) in enumerate(pairs):
        dist_matrix[:, k] = np.linalg.norm(trajs[:, i, :] - trajs[:, j, :], axis=1)

    standalone = ax is None
    if standalone:
        fig, ax = plt.subplots(figsize=(9, 4))

    t_axis = np.arange(T)
    colors_p = plt.cm.Set2(np.linspace(0, 1, len(pairs)))

    for k, (i, j) in enumerate(pairs):
        ax.plot(t_axis, dist_matrix[:, k], '-', color=colors_p[k],
                alpha=0.55, lw=1.0, label=f'P{i+1}–P{j+1}')

    ax.plot(t_axis, dist_matrix.mean(axis=1), 'k-', lw=2, label='Mean dist', zorder=5)
    ax.fill_between(t_axis, dist_matrix.min(axis=1), dist_matrix.max(axis=1),
                    alpha=0.12, color='gray', label='Min–Max range')

    ax.axhline(0, color='red', lw=0.8, ls='--', alpha=0.5)
    ax.set_xlabel('Iteration'); ax.set_ylabel('Distance')
    ax.set_title(title, fontsize=11)
    ax.legend(fontsize=7, ncol=3, loc='upper right')
    ax.grid(True, alpha=0.3)

    if standalone:
        plt.tight_layout()
        plt.show()





Inter-particle distance plot function defined.


### 3.4 Comprehensive Visualization 

We now apply all three visualization tools to each social force model simultaneously. Each column in the figure below corresponds to one model, with rows showing:
- **Row 1:** Trajectories on floor plan
- **Row 2:** Congestion heatmap
- **Row 3:** Inter-particle distances over time

This gives a single unified figure for direct visual comparison.


In [ ]:
from scipy.ndimage import gaussian_filter
import itertools

# ── Re-run simulations (use stored results dict from Task 2) ──────────────────
# We use the 'results' dict computed in Task 2.
# If re-running this cell independently, re-run Task 2 simulation cells first.

social_models_viz = [
    ("No Social Force",     results["No Social Force"]),
    ("Quadratic Repulsion", results["Quadratic Repulsion"]),
    ("Inverse-Distance",    results["Inverse-Distance"]),
    ("Exponential",         results["Exponential"]),
]

fig, axes = plt.subplots(3, 4, figsize=(22, 16))
colors_p = plt.cm.tab10(np.linspace(0, 1, N))

for col, (name, traj) in enumerate(social_models_viz):

    # ── Row 0: Trajectories ───────────────────────────────────────────────────
    ax0 = axes[0, col]
    for a, b in walls:
        ax0.plot([a[0],b[0]], [a[1],b[1]], 'k-', linewidth=2.5)
    for i in range(N):
        ax0.plot(traj[:,i,0], traj[:,i,1], '-', color=colors_p[i], lw=1.5, alpha=0.85)
        ax0.scatter(*starts[i], color=colors_p[i], s=55, edgecolors='k', zorder=5)
        ax0.scatter(*goal_ref, color=colors_p[i], s=90, marker='*', edgecolors='k', zorder=5)
    ax0.set_xlim(-6,6); ax0.set_ylim(-6,6)
    ax0.set_aspect('equal'); ax0.grid(True, alpha=0.2)
    ax0.set_title(f"{name}\n({traj.shape[0]} steps)", fontsize=10, fontweight='bold')
    if col == 0: ax0.set_ylabel("Trajectories", fontsize=10)

    # ── Row 1: Congestion heatmap ─────────────────────────────────────────────
    ax1 = axes[1, col]
    plot_congestion_heatmap(walls, traj, title="", xmin=-6, xmax=6,
                            ymin=-6, ymax=6, bins=80, ax=ax1)
    if col == 0: ax1.set_ylabel("Congestion Heatmap", fontsize=10)

    # ── Row 2: Inter-particle distances ───────────────────────────────────────
    ax2 = axes[2, col]
    plot_interparticle_distances(traj, title="", ax=ax2)
    if col == 0: ax2.set_ylabel("Inter-particle Distance", fontsize=10)

plt.suptitle("Task 3 — Full Visualization Suite: Trajectories, Congestion, and Separation\n"
             "(Baseline Floor Plan, N=5 particles)",
             fontsize=14, y=1.01, fontweight='bold')
plt.tight_layout()
plt.show()


### 3.5 Animations

We now animate the two most contrasting models — **No Social Force** and **Quadratic Repulsion** — so the effect of inter-particle interaction is most clearly visible in motion.


In [ ]:
# ── Animation: No Social Force ────────────────────────────────────────────────
print("Rendering animation: No Social Force...")
anim_none = animate_particles_enhanced(
    walls, starts, goals,
    trajs=results["No Social Force"],
    title="No Social Force — Particles move independently",
    xmin=-6, xmax=6, ymin=-6, ymax=6
)
anim_none


In [ ]:
# ── Animation: Quadratic Repulsion ────────────────────────────────────────────
print("Rendering animation: Quadratic Repulsion...")
anim_quad = animate_particles_enhanced(
    walls, starts, goals,
    trajs=results["Quadratic Repulsion"],
    title="Quadratic Repulsion — Personal space enforced",
    xmin=-6, xmax=6, ymin=-6, ymax=6
)
anim_quad


In [ ]:
# ── Animation: Exponential Social Force ──────────────────────────────────────
print("Rendering animation: Exponential Social Force...")
anim_exp = animate_particles_enhanced(
    walls, starts, goals,
    trajs=results["Exponential"],
    title="Exponential Social Force — Long-range soft repulsion",
    xmin=-6, xmax=6, ymin=-6, ymax=6
)
anim_exp


### 3.6 Analysis and Discussion

**What the visualizations reveal:**

**Trajectory plots** (Row 1) show the overall shape of each particle's path. Without social forces, all trajectories cluster tightly together — they are essentially identical shifted copies. With social forces active, particles fan out and maintain lateral separation even through the corridor.

**Congestion heatmaps** (Row 2) expose bottlenecks that are invisible in trajectory plots. The corridor entrance near $(-1, y)$ and the corner at $(-1, 2)$ are hot spots in all models. With social forces, the heat is more spread out — particles take slightly different routes — whereas without social forces, all particles pile onto the same path, creating a single bright streak.

| Model | Congestion Pattern |
|---|---|
| No Social Force | Single bright streak — all particles on same path |
| Quadratic Repulsion | Spread across ~1–2 corridor widths, diffuse corners |
| Inverse-Distance | Concentrated paths but sharply separated; some near-wall hotspots |
| Exponential | Smoothly distributed, widest spread of the three |

**Inter-particle distance plots** (Row 3) are perhaps the most diagnostic. Without social forces, distances fluctuate freely and can drop to near zero — particles pass through each other. With social forces, a **minimum separation floor** is established. The quadratic model enforces a clean lower bound around $R_s$. The exponential model shows smoother, more gradual separation dynamics.

**Key insight:** The congestion heatmap and distance time-series together reveal a trade-off that the trajectory plot alone cannot: models with stronger repulsion spread particles out more (better separation, lower congestion) but may increase path length and convergence time as particles are forced onto longer detour routes.


---

## Task 4: Comparison of Isotropic and Anisotropic Social Forces

### Overview

All social force models in Tasks 2 and 3 were **isotropic** — the repulsion between two particles depended only on their distance, not on the direction either particle was traveling. This produces a perfectly symmetric force field around each particle.

Real agents, however, are more sensitive to interactions **ahead** of them than behind. A pedestrian pays more attention to someone walking toward them than someone walking away. This directional bias is captured by **anisotropic social forces**, which modulate the interaction strength based on the relative orientation between a particle's velocity and the direction to its neighbor.

The anisotropic social force is defined as:

$$
C^{\text{ani}}_{\text{social}}(i, j) = \left(1 + \beta \cdot \max\left(0,\, \hat{\mathbf{v}}_i \cdot \frac{\mathbf{x}_j - \mathbf{x}_i}{\|\mathbf{x}_j - \mathbf{x}_i\|}\right)\right) \cdot \varphi(\|\mathbf{x}_i - \mathbf{x}_j\|)
$$

where:
- $\hat{\mathbf{v}}_i = \mathbf{v}_i / \|\mathbf{v}_i\|$ is the unit velocity direction of particle $i$
- $\beta \geq 0$ controls the **strength of the directional bias** ($\beta = 0$ recovers the isotropic model)
- $\varphi(d)$ is any scalar distance-based penalty (e.g. quadratic repulsion)

The term $\hat{\mathbf{v}}_i \cdot \frac{\mathbf{x}_j - \mathbf{x}_i}{\|\mathbf{x}_j - \mathbf{x}_i\|}$ is the **cosine of the angle** between particle $i$'s velocity and the direction toward particle $j$. It is:
- **Positive** when $j$ is in front of $i$ → repulsion is amplified
- **Zero** when $j$ is directly to the side
- **Negative** when $j$ is behind $i$ → $\max(0, \cdot)$ clamps it to zero, so no extra penalty

This asymmetry breaks the deadlocks and oscillations that arise when two particles approach each other head-on under isotropic repulsion.


### 4.1 Implementing the Anisotropic Social Force

The key challenge is that the anisotropic model requires the **current velocity** of particle $i$. We estimate this from the previous position:

$$
\mathbf{v}_i^{(k)} \approx \mathbf{x}_i^{(k)} - \mathbf{x}_i^{(k-1)}
$$

We extend `run_multi_particle` to track velocities and pass them to the social force function.


In [ ]:
def social_anisotropic(xi, xj, vi=None, beta=1.5, R_s=1.5, **kwargs):
    
    #Anisotropic (velocity-dependent) social force.

    #C_ani = (1 + beta * max(0, v_hat_i . (xj - xi)/|xj - xi|)) * phi(d)

    d = np.linalg.norm(xi - xj)
    if d < 1e-9:
        return 0.0

    # Base isotropic cost (quadratic band)
    if d > R_s:
        return 0.0
    phi_d = 0.5 * (R_s - d) ** 2

    # Directional modulation
    if vi is None or np.linalg.norm(vi) < 1e-9:
        # No velocity info → fall back to isotropic
        return phi_d

    v_hat = vi / np.linalg.norm(vi)
    e_ij  = (xj - xi) / d          # unit vector from i toward j

    cos_theta = np.dot(v_hat, e_ij)
    modulation = 1.0 + beta * max(0.0, cos_theta)

    return modulation * phi_d


def run_multi_particle_aniso(starts, walls, goals,
                              R, w_wall, w_goal, w_social,
                              phi_wall, beta=1.5, R_s=1.5,
                              alpha=0.001, max_iters=50000, tol=0.05):
   
    N = len(starts)
    positions = [np.array(s, dtype=float) for s in starts]
    prev_pos  = [p.copy() for p in positions]   # for velocity estimation

    if isinstance(goals, np.ndarray) and goals.ndim == 1:
        goals = [goals] * N
    else:
        goals = [np.array(g, dtype=float) for g in goals]

    trajs = [positions.copy()]

    for k in range(max_iters):
        # Estimate current velocities
        velocities = [positions[i] - prev_pos[i] for i in range(N)]

        grads = []
        for i in range(N):
            vi = velocities[i]

            # Goal + wall gradient (reuse existing helpers)
            def ci(x):
                c = goal_cost(x, goals[i], w_goal)
                c += wall_cost(x, walls, R, w_wall, phi_wall)
                for j in range(N):
                    if j != i:
                        c += w_social * social_anisotropic(
                            x, positions[j], vi=vi, beta=beta, R_s=R_s)
                return c

            h = 1e-3
            ex, ey = np.array([1.,0.]), np.array([0.,1.])
            dfdx = (ci(positions[i]+h*ex) - ci(positions[i]-h*ex)) / (2*h)
            dfdy = (ci(positions[i]+h*ey) - ci(positions[i]-h*ey)) / (2*h)
            grads.append(np.array([dfdx, dfdy]))

        prev_pos  = [p.copy() for p in positions]
        positions = [positions[i] - alpha * grads[i] for i in range(N)]
        trajs.append(positions.copy())

        if all(np.linalg.norm(positions[i] - goals[i]) <= tol for i in range(N)):
            print(f"  Converged at iteration {k+1}.")
            break

    return np.array(trajs)




### 4.2 Visualizing the Force Field: Isotropic vs Anisotropic

Before running the full simulation, we visualize the **predicted direction of motion** for a single test particle at various positions, given a fixed velocity direction. This directly mirrors Figure 1.1 from the assignment PDF.

- **Left:** Isotropic — force field is radially symmetric
- **Right:** Anisotropic — force field is asymmetric; repulsion is stronger in front of the moving particle


In [ ]:
def plot_social_field_comparison(phi_iso, phi_ani_fn, beta, R_s,
                                  velocity_dir=np.array([1.0, 0.0]),
                                  xlim=(-4,4), ylim=(-4,4), N=60):
    """
    Side-by-side: isotropic vs anisotropic social force field,
    given a source particle at origin and a fixed velocity direction.
    """
    xs = np.linspace(*xlim, N)
    ys = np.linspace(*ylim, N)
    X, Y = np.meshgrid(xs, ys)
    source = np.array([0.0, 0.0])
    v_fixed = velocity_dir / np.linalg.norm(velocity_dir)

    C_iso = np.zeros_like(X)
    C_ani = np.zeros_like(X)

    for i in range(N):
        for j in range(N):
            xi = np.array([X[i,j], Y[i,j]])
            C_iso[i,j] = phi_iso(xi, source, R_s=R_s)
            C_ani[i,j] = phi_ani_fn(xi, source, vi=v_fixed, beta=beta, R_s=R_s)

    fig, axes = plt.subplots(1, 2, figsize=(13, 5.5))
    titles = [
        "Isotropic Social Force\n(distance-only)",
        f"Anisotropic Social Force\n(β={beta}, velocity →)"
    ]
    for ax, C, title in zip(axes, [C_iso, C_ani], titles):
        dCy, dCx = np.gradient(C, ys, xs)
        cf = ax.contourf(X, Y, C, levels=30, cmap='plasma')
        plt.colorbar(cf, ax=ax, label='Social cost')
        step = 7
        ax.quiver(X[::step,::step], Y[::step,::step],
                  -dCx[::step,::step], -dCy[::step,::step],
                  color='white', alpha=0.65)
        ax.scatter(0, 0, c='red', s=140, zorder=5, label='Source particle')
        # Draw velocity arrow for anisotropic
        if 'Aniso' in title:
            ax.annotate('', xy=(1.5*v_fixed[0], 1.5*v_fixed[1]),
                        xytext=(0, 0),
                        arrowprops=dict(arrowstyle='->', color='cyan', lw=2.5))
            ax.text(1.6*v_fixed[0], 1.6*v_fixed[1], 'v',
                    color='cyan', fontsize=12, fontweight='bold')
        ax.set_title(title, fontsize=12)
        ax.set_xlabel('x'); ax.set_ylabel('y')
        ax.set_aspect('equal')
        ax.legend(fontsize=9)

    plt.suptitle("Social Force Field: Isotropic vs Anisotropic\n"
                 "(Source particle at origin; arrows = push-away direction)",
                 fontsize=12, y=1.02)
    plt.tight_layout()
    plt.show()


# ── Show for rightward velocity ────────────────────────────────────────────────
plot_social_field_comparison(
    social_quadratic, social_anisotropic,
    beta=4.0, R_s=1.5,
    velocity_dir=np.array([1.0, 0.0])
)


### 4.1b  Anisotropic Modulation — 1D Angular Profile and 2D Polar Field

The anisotropic social force scales the base penalty by:

$$
F_{aniso} = 1 + \beta \cdot \max\left(0,\, \cos\theta\right)
$$

where $\theta$ is the angle between particle $i$'s velocity $\hat{\mathbf{v}}_i$ and the direction toward neighbor $j$.

This is analogous to the **joint-range limit function** in the reference notes: a 1D curve that is non-uniform over its domain (here $\theta \in [0°, 360°]$), with a region of elevated penalty (forward hemisphere) and a flat base elsewhere. We plot this modulation function and its induced 2D asymmetric field for several values of $\beta$.


In [ ]:


fig, axes = plt.subplots(1, 1, figsize=(14, 5))
ax1 = axes

# ── Left: M(theta) vs theta in degrees ───────────────────────────────────────
theta_deg = np.linspace(0, 360, 720)
theta_rad = np.deg2rad(theta_deg)
colors_b  = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd']
beta_vals  = [0.0, 0.5, 1.0, 2.0, 3.0]

for beta, col in zip(beta_vals, colors_b):
    M = 1.0 + beta * np.maximum(0.0, np.cos(theta_rad))
    ax1.plot(theta_deg, M, lw=2, color=col, label=f'$\\beta = {beta}$')

# Annotate forward/backward hemispheres
ax1.axvspan(0,   90,  alpha=0.06, color='green')
ax1.axvspan(270, 360, alpha=0.06, color='green')
ax1.axvspan(90,  270, alpha=0.06, color='red')
# ax1.text( 10, 0.15, 'Forward\nhemisphere', color='green', fontsize=8, va='bottom')
# ax1.text(120, 0.15, 'Rear\nhemisphere',   color='red',   fontsize=8, va='bottom')
ax1.axvline(90,  color='gray', lw=1, ls='--', alpha=0.5)
ax1.axvline(270, color='gray', lw=1, ls='--', alpha=0.5)
ax1.set_xlabel('Angle $\\theta$ (degrees) — direction to neighbor', fontsize=11)
ax1.set_ylabel(' $ 1 + \\beta \\max(0, \\cos\\theta)$', fontsize=11)
ax1.set_title('Anisotropic Modulation Factor vs. Angle\n(velocity direction = 0°)', fontsize=11)
ax1.legend(fontsize=9, loc='upper right'); ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 360); ax1.set_xticks([0,90,180,270,360])
ax1.set_xticklabels(['0°\n(ahead)','90°','180°\n(behind)','270°','360°'])


plt.tight_layout(); plt.show()


The above  plot reveal a key property: when $\beta = 0$ the curve is flat same as the isotropic model. As $\beta$ increases, the forward lobe (green region, $\theta < 90°$ and $\theta > 270°$) grows while the rear hemisphere stays at the same postion as $\beta = 0$ . 




In [ ]:


beta_show = [0.0, 1.0, 2.0, 3.0]
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

N_g = 80
lim = 2.5
xs = np.linspace(-lim, lim, N_g)
ys = np.linspace(-lim, lim, N_g)
X, Y = np.meshgrid(xs, ys)
source = np.array([0.0, 0.0])
v_fixed = np.array([1.0, 0.0])   # moving rightward

for ax, beta in zip(axes, beta_show):
    Z = np.zeros_like(X)
    for i in range(N_g):
        for j in range(N_g):
            Z[i,j] = social_anisotropic(
                np.array([X[i,j], Y[i,j]]), source,
                vi=v_fixed, beta=beta, R_s=1.8
            )
    dZy, dZx = np.gradient(Z, ys, xs)
    cf = ax.contourf(X, Y, Z, levels=25, cmap='plasma')
    plt.colorbar(cf, ax=ax, label='$C^{\\mathrm{ani}}$')
    step = 8
    ax.quiver(X[::step,::step], Y[::step,::step],
              -dZx[::step,::step], -dZy[::step,::step],
              color='white', alpha=0.6)
    # Influence radius circle
    theta_c = np.linspace(0, 2*np.pi, 200)
    ax.plot(1.8*np.cos(theta_c), 1.8*np.sin(theta_c), 'w--', lw=1.2, alpha=0.6)
    ax.scatter(0, 0, c='red', s=120, zorder=5)
    ax.annotate('', xy=(1.2, 0), xytext=(0, 0),
                arrowprops=dict(arrowstyle='->', color='cyan', lw=2.2))
    ax.set_xlim(-lim, lim); ax.set_ylim(-lim, lim)
    ax.set_aspect('equal')
    ax.set_title(f'$\\beta = {beta}$\n({"isotropic" if beta==0 else "anisotropic"})',
                 fontsize=11)
    ax.set_xlabel('$x$'); ax.set_ylabel('$y$')

plt.suptitle('2D Anisotropic Social Force Field for Increasing $\\beta$\n'
             '(source at origin, particle velocity → rightward, $R_s=1.8$)',
             fontsize=12, y=1.02)
plt.tight_layout(); plt.show()


#### Effect of $\beta$ on Field Asymmetry

The four panels show the progressive transition from **isotropic** ($\beta=0$, perfectly circular cost contours) to **strongly anisotropic** ($\beta=3$, heavily forward-biased field). The cyan arrow indicates the particle's velocity direction.

| $\beta$ | Field shape | Gradient behavior |
|---|---|---|
| 0 | Circular (isotropic) | Symmetric push in all directions |
| 1 | Slight forward elongation | Mild forward bias |
| 2 | Clearly asymmetric lobe | Strong forward deflection |
| 3 | Dominant forward lobe | Particle strongly avoids what is ahead |

The white dashed circle marks $R_s$ — the base influence radius of the underlying quadratic penalty. Notice that even within $R_s$, the cost is non-uniform: the region in front of the particle is brighter (higher cost) than behind it.


### 1D Profile & 3D Surface — Anisotropic Social Force

**Top row (1D):** Forward-axis penalty (red solid) vs. rear-axis penalty (teal dashed). When $\beta=0$ both curves coincide (isotropic). As $\beta$ increases, the forward penalty grows by factor $(1+\beta)$ while the rear stays at the base $\varphi(d)$.

**Bottom row (3D surface):** $F(x,y)$ field around the source particle (at origin), with the particle moving rightward. The surface deforms from a **symmetric cone** ($\beta=0$) into an **asymmetric forward lobe** as $\beta$ increases — particles are more repelled by neighbors in front of them.


In [ ]:
def plot_aniso_1d_and_3d(beta_vals=[0.0, 1.5, 3.0], R_s=1.8, N_g=80, lim=2.5):
    """
    For each beta:
      - 1D profile along forward axis vs rear axis
      - 3D surface of the anisotropic cost field
    Particle velocity is fixed rightward [1, 0].
    """
    v_fixed = np.array([1.0, 0.0])
    source  = np.array([0.0, 0.0])
    xs = np.linspace(-lim, lim, N_g)
    ys = np.linspace(-lim, lim, N_g)
    X, Y = np.meshgrid(xs, ys)

    n = len(beta_vals)
    fig = plt.figure(figsize=(6*n, 9))

    d_line = np.linspace(1e-3, lim, 400)

    for col, beta in enumerate(beta_vals):
        Z = np.zeros_like(X)
        for i in range(N_g):
            for j in range(N_g):
                Z[i,j] = social_anisotropic(
                    np.array([X[i,j], Y[i,j]]), source,
                    vi=v_fixed, beta=beta, R_s=R_s
                )

        # ── 1D cross-sections (top row) ───────────────────────────────────────
        ax1 = fig.add_subplot(2, n, col+1)
        fwd  = [social_anisotropic(np.array([d,  0.]), source, vi=v_fixed, beta=beta, R_s=R_s) for d in d_line]
        rear = [social_anisotropic(np.array([-d, 0.]), source, vi=v_fixed, beta=beta, R_s=R_s) for d in d_line]
        ax1.plot(d_line, fwd,  lw=2.2, color='#e84a5f', label='Forward ($\\theta=0°$)')
        ax1.plot(d_line, rear, lw=2.2, color='#2a9d8f', ls='--', label='Rear ($\\theta=180°$)')
        ax1.axvline(R_s, color='gray', lw=1, ls='--', alpha=0.6)
        ax1.text(R_s+0.05, 0.01, f'$R_s$', color='gray', fontsize=9)
        ax1.set_xlabel('distance (d)', fontsize=10)
        ax1.set_ylabel('$C^{\\mathrm{ani}}(d)$', fontsize=10)
        ax1.set_title(f'$\\beta={beta}$ — 1D Profile\n(fwd vs rear penalty)', fontsize=10)
        ax1.legend(fontsize=8); ax1.grid(True, alpha=0.3)
        ax1.set_xlim(0, lim); ax1.set_ylim(bottom=0)

        # ── 3D surface (bottom row) ────────────────────────────────────────────
        ax2 = fig.add_subplot(2, n, n+col+1, projection='3d')
        ax2.plot_surface(X, Y, Z, cmap='plasma', linewidth=0, antialiased=True, alpha=0.92)
        ax2.set_xlabel('x', fontsize=8); ax2.set_ylabel('y', fontsize=8)
        ax2.set_zlabel('F(x,y)', fontsize=8)
        ax2.set_title(f'$\\beta={beta}$ — 3D Field\n({"isotropic" if beta==0 else "anisotropic, v→"})', fontsize=10)
        ax2.view_init(elev=28, azim=-55)

    plt.suptitle('Anisotropic Social Force \n'
                 '(velocity direction = rightward →; $R_s={:.1f}$)'.format(R_s),
                 fontsize=13, fontweight='bold', y=1.01)
    plt.tight_layout()
    plt.show()


plot_aniso_1d_and_3d(beta_vals=[0.0, 1.5, 5.0], R_s=1.8)


**Key observations from the force field plots:**

The isotropic field (left) is perfectly radially symmetric — every direction around the source particle receives equal repulsion. The anisotropic field (right) breaks this symmetry: the repulsion **in front of the moving particle** (in the direction of $\mathbf{v}$) is amplified by the factor $(1 + \beta)$, while the repulsion **behind** the particle is unchanged at its base $\varphi(d)$ value.

This asymmetry has an important practical consequence: when two particles approach each other head-on, the isotropic model pushes them equally in both lateral directions, which can cause them to oscillate. The anisotropic model biases each particle to step sideways relative to its own direction of travel, naturally producing **lane-like behavior** similar to pedestrian crowd dynamics.


### 4.3 Multi-Particle Simulation — Isotropic vs Anisotropic

We now run the full simulation on the **baseline** floor plan with:
1. **Isotropic Quadratic Repulsion** (β = 0, pure distance-based)
2. **Anisotropic Social Force** at three β values: 0.5, 1.5, 3.0

All other parameters are held constant for a clean comparison.


In [ ]:
# ── Shared parameters ─────────────────────────────────────────────────────────
walls, start_ref, goal_ref = load_floor_plan("baseline")
N = 5
np.random.seed(2026)
starts_ani = [(-2,1.5),(-3,-1.5),(-4,-4),(-4,1),(-3,1)]
goals_ani  = [goal_ref] * N

R, w_wall, w_goal, w_social = 1.0, 30.0, 2.0, 5.0
alpha = 0.001

# ── Isotropic baseline ────────────────────────────────────────────────────────
print("Running: Isotropic (β=0)...")
traj_iso = run_multi_particle(
    starts_ani, walls, goals_ani,
    R=R, w_wall=w_wall, w_goal=w_goal, w_social=w_social,
    phi_wall=quadratic_band_penalty,
    phi_social=social_quadratic,
    alpha=alpha, max_iters=50000, tol=0.05,
    R_s=1.5
)

# ── Anisotropic at different β values ─────────────────────────────────────────
aniso_results = {}
for beta_val in [0.5, 1.5, 5.0]:
    print(f"Running: Anisotropic β={beta_val}...")
    traj = run_multi_particle_aniso(
        starts_ani, walls, goals_ani,
        R=R, w_wall=w_wall, w_goal=w_goal, w_social=w_social,
        phi_wall=quadratic_band_penalty,
        beta=beta_val, R_s=1.5,
        alpha=alpha, max_iters=50000, tol=0.05
    )
    aniso_results[beta_val] = traj



In [ ]:
# ── 4-panel trajectory comparison ─────────────────────────────────────────────
all_runs = [
    ("Isotropic (β=0)",    traj_iso),
    ("Anisotropic β=0.5",  aniso_results[0.5]),
    ("Anisotropic β=1.5",  aniso_results[1.5]),
    ("Anisotropic β=3.0",  aniso_results[5.0]),
]

fig, axes = plt.subplots(1, 4, figsize=(22, 6))
colors_n = plt.cm.tab10(np.linspace(0, 1, N))

for ax, (name, traj) in zip(axes, all_runs):
    for a, b in walls:
        ax.plot([a[0],b[0]], [a[1],b[1]], 'k-', linewidth=2.5)
    for i in range(N):
        ax.plot(traj[:,i,0], traj[:,i,1], '-', color=colors_n[i], lw=1.5, alpha=0.85)
        ax.scatter(*starts_ani[i], color=colors_n[i], s=55, edgecolors='k', zorder=5)
        ax.scatter(*goal_ref, color=colors_n[i], s=90, marker='*', edgecolors='k', zorder=5)
    ax.set_xlim(-6,6); ax.set_ylim(-6,6)
    ax.set_aspect('equal'); ax.grid(True, alpha=0.2)
    ax.set_title(f"{name}\n({traj.shape[0]} steps)", fontsize=10, fontweight='bold')
    ax.set_xlabel('x'); ax.set_ylabel('y')

plt.suptitle("Task 4 — Isotropic vs Anisotropic Social Forces (Baseline, N=5)",
             fontsize=13, y=1.01, fontweight='bold')
plt.tight_layout()
plt.show()


### 4.4 Quantitative Comparison: Separation, Path Length, Convergence


In [49]:
print(f"{'Model':<25} {'Steps':>8} {'Mean Min Dist':>15} {'Avg Path Len':>14} {'Max Near-Miss':>15}")
print("-" * 80)

for name, traj in all_runs:
    T_run, N_run, _ = traj.shape
    pairs_run = list(itertools.combinations(range(N_run), 2))

    dist_mat = np.array([
        [np.linalg.norm(traj[t,i,:] - traj[t,j,:]) for i,j in pairs_run]
        for t in range(T_run)
    ])

    mean_min = dist_mat.min(axis=1).mean()
    near_miss = dist_mat.min()               # global minimum distance (worst case)
    path_len = np.mean([
        np.sum(np.linalg.norm(np.diff(traj[:,i,:], axis=0), axis=1))
        for i in range(N_run)
    ])
    print(f"{name:<25} {T_run:>8}  {mean_min:>14.3f}  {path_len:>13.3f}  {near_miss:>14.3f}")


Model                        Steps   Mean Min Dist   Avg Path Len   Max Near-Miss
--------------------------------------------------------------------------------
Isotropic (β=0)              15001           0.737          9.391           0.664
Anisotropic β=0.5            15001           0.832         19.267           0.732
Anisotropic β=1.5            15001           0.951         34.186           0.828
Anisotropic β=3.0            15001           1.071         50.077           0.931


### 4.6 Congestion Comparison


In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(22, 6))

for ax, (name, traj) in zip(axes, all_runs):
    plot_congestion_heatmap(walls, traj, title=name, ax=ax)

plt.suptitle("Task 4 — Congestion Heatmaps: Isotropic vs Anisotropic",
             fontsize=13, y=1.01, fontweight='bold')
plt.tight_layout()
plt.show()


### 4.7 Animations: Isotropic vs Anisotropic


In [ ]:
# ── Isotropic animation ───────────────────────────────────────────────────────
print("Rendering: Isotropic...")
animate_particles_enhanced(
    walls, starts_ani, goals_ani, traj_iso,
    title="Isotropic Quadratic Repulsion (β=0)",
    xmin=-6, xmax=6, ymin=-6, ymax=6
)


In [ ]:
# ── Anisotropic β=1.5 animation ───────────────────────────────────────────────
print("Rendering: Anisotropic β=5.0...")
animate_particles_enhanced(
    walls, starts_ani, goals_ani, aniso_results[5.0],
    title="Anisotropic Social Force (β=5.0)",
    xmin=-6, xmax=6, ymin=-6, ymax=6
)


### 4.8 Zigzag Floor Plan 

Narrow corridors are where the isotropic vs. anisotropic distinction matters most. When two groups of particles must pass through the same corridor in the same direction, the isotropic model may produce oscillations. We test this on the zigzag floor plan.


In [ ]:
walls_zz, start_zz, goal_zz = load_floor_plan("zigzag")
N_zz = 5
starts_zz2 = [(-3,-3.5),(-5,-1),start_zz,(-1.8,-3),(-4,2) ]
goals_zz2  = [goal_zz] * N_zz

print("Running zigzag: Isotropic...")
traj_zz_iso = run_multi_particle(
    starts_zz2, walls_zz, goals_zz2,
    R=1.0, w_wall=20.0, w_goal=2.0, w_social=5.0,
    phi_wall=quadratic_band_penalty,
    phi_social=social_quadratic,
    alpha=0.001, max_iters=50000, tol=0.05, R_s=1.0
)

print("Running zigzag: Anisotropic β=1.5...")
traj_zz_ani = run_multi_particle_aniso(
    starts_zz2, walls_zz, goals_zz2,
    R=1.0, w_wall=20.0, w_goal=2.0, w_social=5.0,
    phi_wall=quadratic_band_penalty,
    beta=1.5, R_s=1.0,
    alpha=0.001, max_iters=50000, tol=0.05
)

# ── Side-by-side ──────────────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
colors_zz = plt.cm.tab10(np.linspace(0, 1, N_zz))

for ax, (name, traj) in zip(axes, [("Isotropic", traj_zz_iso),
                                    ("Anisotropic β=1.5", traj_zz_ani)]):
    for a, b in walls_zz:
        ax.plot([a[0],b[0]], [a[1],b[1]], 'k-', linewidth=2.5)
    for i in range(N_zz):
        ax.plot(traj[:,i,0], traj[:,i,1], '-', color=colors_zz[i], lw=1.5, alpha=0.85)
        ax.scatter(*starts_zz2[i], color=colors_zz[i], s=60, edgecolors='k', zorder=5)
        ax.scatter(*goal_zz, color=colors_zz[i], s=100, marker='*', edgecolors='k', zorder=5)
    ax.set_xlim(-7,7); ax.set_ylim(-5,5)
    ax.set_aspect('equal'); ax.grid(True, alpha=0.25)
    ax.set_title(f"{name} — Zigzag Floor Plan\n({traj.shape[0]} steps)", fontsize=11)

plt.suptitle("Task 4 — Zigzag Corridor: Isotropic vs Anisotropic (N=4)",
             fontsize=13, y=1.02, fontweight='bold')
plt.tight_layout()
plt.show()


### 4.9 Comparative Analysis

**1. How does directional dependence affect the symmetry of the social force field?**

The isotropic model produces a perfectly radially symmetric force field around each particle. Every neighbor, regardless of direction, is repelled equally. The anisotropic model breaks this symmetry by scaling the repulsion by $(1 + \beta \cos\theta)$ where $\theta$ is the angle between the particle's velocity and the direction to its neighbor. The result is an **asymmetric, forward-biased** repulsion field. Neighbors directly ahead receive up to $(1 + \beta)$ times the base repulsion, while neighbors behind receive only the unmodified base cost. This is visible in the force field plots of Section 4.2.

**2. Does the anisotropic model reduce oscillations or deadlocks? Why?**

Yes, in most configurations. The isotropic model can produce **symmetric deadlocks**: two particles approaching each other head-on experience equal lateral push in both directions, causing them to either pass through or oscillate. The anisotropic model resolves this because each particle is more strongly repelled by whatever it is moving toward, biasing it to deflect sideways relative to its own trajectory. This naturally produces **lane formation** — an emergent collective behavior where particles traveling in the same direction align into streams, reducing cross-traffic.

**3. How does each model behave in narrow corridors or near bottlenecks?**

In narrow corridors (zigzag floor plan), the isotropic model forces particles into a tight queue with relatively uniform spacing governed by $R_s$. The anisotropic model tends to produce a more **ordered single file** because each trailing particle is more aware of the particle ahead (which is in its forward cone) and backs off faster. Near bottlenecks (corridor corners), the anisotropic model can also resolve crowding faster because particles entering the bottleneck are more sensitive to congestion ahead.

**4. What trade-offs arise when introducing velocity-dependent interactions?**

The main cost of anisotropy is increased sensitivity to parameter choices. A large $\beta$ can cause particles to over-react to neighbors in their path, producing sharp turns and increased path length. The optimal $\beta$ depends on the environment geometry and particle density.

**Summary:**

The anisotropic social force model represents a meaningful improvement over the isotropic model in terms of realism and deadlock resolution. The directional bias — prioritizing interactions in the forward hemisphere — mirrors how real agents perceive and react to their environment. However, it comes at the cost of additional parameter tuning and slightly less predictable behavior when particles are slow-moving or stationary. For most multi-agent navigation scenarios, a moderate $\beta \approx 1.0$–$2.0$ combined with quadratic repulsion provides the best balance of stability, realism, and separation quality.
